In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PreprocessingUtil as util

In [2]:
data = pd.read_csv('Processed_Transaction_Narations.csv')

In [3]:
narations = data['Processed_Naration'].dropna().unique()

In [4]:
#tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(smooth_idf=True, use_idf=True, min_df = 5)
vectorizer.fit_transform(narations)
feature_names = vectorizer.get_feature_names()

In [5]:
TOP_K_KEYWORDS = 10
result = []
for doc in narations:
    df = {}
    df['processed_narations'] = doc
    df['top_keywords'] = util.get_keywords(vectorizer, feature_names, doc, TOP_K_KEYWORDS)
    result.append(df)
final = pd.DataFrame(result)

In [13]:
from gensim.models import Word2Vec
final['top_keywords'] = final['processed_narations']
model = Word2Vec(final['top_keywords'], min_count=1, vector_size = 150)

In [15]:
final['avg_vec'] = final['top_keywords'].apply(lambda text : util.mean_word2vec(text,model))

In [16]:
#kmeans clustering
from sklearn.cluster import KMeans

In [ ]:
wcss = []
for i in range(7,15):
    kmeans = KMeans(i, init = 'k-means++', max_iter = 500,n_init = 10, random_state = 0)
    kmeans.fit(list(final['avg_vec']))
    wcss.append(kmeans.inertia_)

In [ ]:
plt.plot(range(7,15), wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [17]:
kmeans = KMeans(n_clusters = 15, init = 'k-means++', max_iter = 500, n_init=10, random_state=0)

In [18]:
final['kmeans_cluster'] = kmeans.fit_predict(list(final['avg_vec']))

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

In [ ]:
# determining value of eps for dbscan
neighbors = NearestNeighbors(n_neighbors=20)
neighbors_fit = neighbors.fit(list(final['avg_vec']))
distances, indices = neighbors_fit.kneighbors(list(final['avg_vec']))
distances = np.sort(distances, axis=0)
distances = distances[:,1]
plt.plot(distances)

In [ ]:
db = DBSCAN(eps=0.9, min_samples=300).fit(list(final['avg_vec']))
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

In [ ]:
final['dbscan_cluster']  = labels

In [ ]:
from sklearn.cluster import MiniBatchKMeans

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=12, random_state=0, batch_size=100)

In [ ]:
final["minibatch_cluster"] = kmeans.fit_predict(list(final['avg_vec']))

In [ ]:
final.to_excel('results.xlsx')